# Capítulo 6 - Reward Shaping e Curriculum Learning


Vamos praticar essas duas técnicas no ambiente **"MountainCar-v0"** do gym.

Você pode rodar este notebook no Colab ou localmente. Para abrir diretamente no Colab, basta clicar no botão abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap06/cap06-other-techniques.ipynb) 

Para rodar localmente, primeiro, baixe todo o repositório do github: https://github.com/pablo-sampaio/rl_facil.

## 1. Configurações Iniciais

### Cria Diretório para Experimentos

In [1]:
!mkdir results

J� existe uma subpasta ou um arquivo results.


### Configurações Dependentes do Sistema

Rode a célula abaixo, mesmo sem estar no Colab.

In [10]:
import sys
from IPython.display import clear_output

if 'google.colab' in sys.modules:
    !pip install gym
    !pip install gym[box2d]
    !pip install optuna

    # para salvar videos
    !apt-get install -y xvfb x11-utils
    !pip install pyvirtualdisplay==0.2.*
    !apt-get install ffmpeg

    from pyvirtualdisplay import Display
    global display
    display = Display(visible=False, size=(1400, 900))
    _ = display.start()

    !git clone https://github.com/pablo-sampaio/rl_facil
    clear_output()

    !mv /content/rl_facil/cap04/* /content/
    !mv /content/rl_facil/cap05/* /content/
    !mv /content/rl_facil/cap06/* /content/

else:
    # atenção: para Windows apenas! 
    # se estiver no Linux, troque por "copy" por "cp" e mude a barra
    !copy ..\cap04\util_*.py .


..\cap04\util_experiments.py
..\cap04\util_plot.py
        2 arquivo(s) copiado(s).


### Configurações para Exibir Video

In [3]:
# ideias adaptadas de : https://www.anyscale.com/blog/an-introduction-to-reinforcement-learning-with-openai-gym-rllib-and-google
from base64 import b64encode
from IPython.display import HTML
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  html_code = f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'
  return HTML(html_code)

### Imports Principais

Import algoritmos, ambientes e outros módulos.

In [2]:
import gym
import numpy as np
import optuna

from util_experiments import repeated_exec
from util_plot import plot_result, plot_multiple_results

from models_torch import test_policy
#from crossentropy_method_v1 import run_crossentropy_method1, PolicyModelCrossentropy
from crossentropy_method_v2 import run_crossentropy_method2, PolicyModelCrossentropy

## 2. Modelagem de Recomepensas (Reward Shaping)

Na **modelagem de recompensas**, nós alteramos o modelo de recompensa do ambiente original para dar recompensas mais *informativa*.

Primeiro, criamos um wrapper do ambiente, em que damos recompensas adicionais a cada novo avanço conseguido pelo algoritmo na direção `x`.

Veja os detalhes do estado do MountainCar em https://www.gymlibrary.ml/environments/classic_control/mountain_car/ .

Ideia:
- Calcular o deslocamento `delta` em relação ao `x` inicial (pode ser para a esquerda ou direita)
- Toda vez que o agente ultrapassa o `delta` máximo obtido no episódio, ele recebe uma recompensa extra.
- Se atingir o alvo, recebe uma recompensa alta
- Atenção: pode estar violando a especificação de MDP

In [7]:
class FrequentRewardsMC(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def reset(self):
        obs = self.env.reset()
        self.dist = 0
        self.initial_pos = obs[0]
        self.max_delta = 0
        return obs
    
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        delta = abs(obs[0] - self.initial_pos) # quão distante está da posição inicial
        if delta > self.max_delta:
            reward = delta
        if obs[0] >= 0.6:
            reward = 100.0
        return obs, reward, done, info


In [11]:
rmax = 50.0
ENV = gym.make("MountainCar-v0")
ENV = FrequentRewardsMC(ENV)

EPISODES   = 2000    # total de episódios
BATCH_SIZE = 50      # quantidade de episódios executados por época de treinamento
PROPORTION = 0.1     # percentual dos episódios (do batch) que serão selecionados

policy_model = PolicyModelCrossentropy(ENV.observation_space.shape[0], [512], ENV.action_space.n, lr=0.005)
returns, policy = run_crossentropy_method2(ENV, EPISODES, BATCH_SIZE, PROPORTION)
clear_output()

print("Últimos episódios do treinamento: media =", np.mean(returns[-20:]), ", desvio padrao =", np.std(returns[-20:]))


Últimos episódios do treinamento: media = 21.816432578861715 , desvio padrao = 10.571414522501259


In [ ]:
# Exibe um gráfico episódios x retornos (não descontados)
plot_result(returns, rmax)

In [ ]:
# Faz alguns testes com o modelo de forma DETERMINÍSTICA e salva o vídeo em arquivo
video = VideoRecorder(ENV, "politica-treinada.mp4")
test_policy(ENV, policy, True, 5, render=False, videorec=video)

In [ ]:
render_mp4("politica-treinada.mp4")

## 3. Aprendizagem por Currículo (Curriculum Learning)


Na *aprendizagem por currículo* aplicada em RL:
- o agente é treinado sequencialmente em *vários* ambientes (parecidos)
- os ambientes têm níveis crescentes de dificuldade
- ele deve aprender bem um deles antes de passar para o próximo

Abaixo, criamos um wrapper que permite instanciar **versões simplificadas do MountainCar**.
- a nova versão é parametrizada por um valor `goal_x`
- basta o agente ultrapassar essa posição com velocidade positiva para terminar o episódio
- a recompensa final é a velocidade (quanto mais rápido ele passar, melhor)
- se `goal_x > 0.6`, ele funciona idêntico ao ambiente original

In [15]:
class SimplifiedMC(gym.Wrapper):
    def __init__(self, env, goal_x):
        super().__init__(env)
        self.goal_x = goal_x

    def reset(self):
        obs = self.env.reset()
        return obs
    
    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        if obs[0] >= self.goal_x and obs[1] > 0:
            reward = obs[1]
            done = True
        return obs, reward, done, info

Para treinar, vamos usar uma rede com DUAS camadas intermediárias: 128 e 256.

E vamos treinar em diferentes instâncais do ambiente acima, com valores de `goal_x` crescentes.

In [ ]:
rmax = 50.0
ENV = gym.make("MountainCar-v0")

BATCH_SIZE   = 50     # quantidade de episódios executados por época de treinamento
PROPORTION = 0.1      # percentual dos episódios (do batch) que serão selecionados

policy = PolicyModelCrossentropy(ENV.observation_space.shape[0], [128, 256], ENV.action_space.n, lr=0.01)

all_returns = []

curriculum_params = [(-0.30, 5000, -90.0), (-0.25, 5000, -90.0), (-0.20, 5000, -100.0), (-0.15, 5000, -100.0), (-0.10, 5000, -110.0), (-0.05, 5000, -120.0), (0.00, 5000, -120.0), (0.7, 5000, -80.0)]

for goal_x, episodes, target_return in curriculum_params:
    print(f"TREINANDO COM goal_x = {goal_x}:")
    wrapped_env = SimplifiedMC(ENV, goal_x)
    
    returns, policy = run_crossentropy_method2(wrapped_env, episodes, BATCH_SIZE, PROPORTION, target_return=target_return, initial_policy=policy, render=False)
    print("Últimos resultados: media =", np.mean(returns[-20:]), ", desvio padrao =", np.std(returns[-20:]))        
    plot_result(returns, rmax)
    all_returns.extend(returns)

plot_result(all_returns, rmax)



In [ ]:
# Executa alguns episódios de forma NÃO-determinística e imprime um sumário
test_policy(ENV, policy, False, 5, render=True)